In [1]:
import uproot
import awkward as ak
import numpy as np
import vector
vector.register_numba()
vector.register_awkward()


In [2]:
columns = [
    "era",
    "higgs1_reco_pt",
    "higgs1_reco_eta",
    "higgs1_reco_phi",
    "higgs1_reco_mass",
    "higgs2_reco_pt",
    "higgs2_reco_eta",
    "higgs2_reco_phi",
    "higgs2_reco_mass",
    "HT",
    "higgs1_DeltaRjj",
    "higgs2_DeltaRjj",
    "minDeltaR_Higgjj",
    "maxDeltaR_Higgjj",
    "higgs1_helicityCosTheta",
    "higgs2_helicityCosTheta",
    "hh_CosThetaStar_CS",
    "hh_vec_mass",
    "hh_vec_pt",
    "hh_vec_eta",
    "hh_vec_DeltaR",
    "hh_vec_DeltaPhi",
    "hh_vec_DeltaEta",
    "higgs1_reco_jet1_pt",
    "higgs1_reco_jet1_eta",
    "higgs1_reco_jet1_phi",
    "higgs1_reco_jet1_mass",
    "higgs1_reco_jet2_pt",
    "higgs1_reco_jet2_eta",
    "higgs1_reco_jet2_phi",
    "higgs1_reco_jet2_mass",
    "higgs2_reco_jet1_pt",
    "higgs2_reco_jet1_eta",
    "higgs2_reco_jet1_phi",
    "higgs2_reco_jet1_mass",
    "higgs2_reco_jet2_pt",
    "higgs2_reco_jet2_eta",
    "higgs2_reco_jet2_phi",
    "higgs2_reco_jet2_mass",
    "add_jet1pt_pt",
    "add_jet1pt_eta",
    "add_jet1pt_phi",
    "add_jet1pt_mass",
    "sigma_over_higgs1_reco_mass",
    "sigma_over_higgs2_reco_mass",
]
main_dir='/pnfs/psi.ch/cms/trivcat/store/user/mmalucch/keras_models_morphing'


In [6]:
input_root=f'/pnfs/psi.ch/cms/trivcat/store/user/mmalucch/JetMET_2022EE_2b_signal_region_to_4b_soumya_january2025.root'
tree=uproot.open(input_root)["tree"]
print(tree.keys())
for k in tree.keys():
    if "res" in k:
        print(k)
    if "index" in k or "idx" in k:
        print(k)


['era', 'event', 'HT', 'jet_pt', 'jet_pt_raw', 'jet_eta', 'jet_phi', 'jet_mass', 'jet_mass_raw', 'jet_deepjet_probBVsAll', 'jet_pnet_probBVsAll', 'jet_pnet_probTauVsJet', 'jet_pnet_probb', 'jet_pnet_probc', 'jet_pnet_probuds', 'jet_pnet_probtauh', 'jet_pnet_ptcorr', 'jet_pnet_ptnu', 'jet_pnet_ptres', 'jet_pnet_jec_L1L2L3', 'jet_pnet_jec_L2Rel', 'jet_pnet_jec_L2L3Res', 'btag_cat', 'weight', 'is_boosted', 'weight_dnn', 'sigma_higgs1', 'sigma_over_higgs1_reco_mass', 'higgs1_reco_pt', 'higgs1_reco_eta', 'higgs1_reco_phi', 'higgs1_reco_mass', 'sigma_higgs2', 'sigma_over_higgs2_reco_mass', 'higgs2_reco_pt', 'higgs2_reco_eta', 'higgs2_reco_phi', 'higgs2_reco_mass', 'hh_vec_mass', 'hh_vec_pt', 'hh_vec_eta', 'hh_vec_phi', 'hh_vec_DeltaR', 'hh_vec_DeltaPhi', 'hh_vec_DeltaEta', 'hh_vec_ptOmass', 'njet', 'higgs1_reco_jet1_pt', 'higgs1_reco_jet1_eta', 'higgs1_reco_jet1_phi', 'higgs1_reco_jet1_mass', 'higgs1_reco_jet2_pt', 'higgs1_reco_jet2_eta', 'higgs1_reco_jet2_phi', 'higgs1_reco_jet2_mass', 'hig

get the columns from the tree

In [105]:
n_events = 3
events = tree.arrays(tree.keys(), library="ak")[:n_events]
print(events[:n_events] , type(events.era), len(events['era']))


[{era: 1, event: 688250681, HT: 321, ... add_jet1pt_Higgs2_m: 223}] <class 'awkward.highlevel.Array'> 3


In [113]:
print(events.jet_pnet_ptres[2])
print(events.jet_pt[2])
print(events.higgs1_reco_jet1_pt[2])
print(events.higgs1_reco_jet2_pt[2])
print(events.higgs2_reco_jet1_pt[2])
print(events.higgs2_reco_jet2_pt[2])

[0.0833, 0.107, 0.0626, 0.12, 0.194]
[123, 82.9, 79.4, 76.9, 32]
122.9034652709961
76.88542938232422
79.41437530517578
32.01162338256836


In [114]:
# given his snippet of code in c++ translate in python using akward arrays and vectors

# float HelicityCosTheta(TLorentzVector Booster, TLorentzVector Boosted)
# {
#     TVector3 BoostVector = Booster.BoostVector();
#     Boosted.Boost( -BoostVector.x(), -BoostVector.y(), -BoostVector.z() );
#     return Boosted.CosTheta();
# }

# higgs1_helicityCosTheta =fabs(HelicityCosTheta( leadingHiggsCands.at(best_pairing_index)  ,  leading_higgs_leading_jet ));
# higgs2_helicityCosTheta =fabs(HelicityCosTheta( subleadingHiggsCands.at(best_pairing_index), subleading_higgs_leading_jet ));

# //Costhetastar in CS frame
# cosine of the polar angle of one Higgs boson with respect to the beam axis in the HH rest frame
# TLorentzVector higgs1_vec;
# higgs1_vec =  leadingHiggsCands.at(best_pairing_index);
# higgs1_vec.Boost( - hh_vec.BoostVector());
# hh_CosThetaStar_CS = fabs(higgs1_vec.CosTheta());


def HelicityCosTheta(higgs, jet):
    # print("jet px",jet.px)
    # print("higgs px",higgs.px)
    # print("jet theta",jet.theta)
    higgs_velocity = higgs.to_beta3()
    jet = jet.boost_beta3(-higgs_velocity)
    # print("jet px",jet.px)
    # print("jet theta",jet.theta)
    return np.cos(jet.theta)


def Costhetastar_CS(higgs1_vec, hh_vec):
    # print("px",higgs1_vec.px)
    # print("theta",higgs1_vec.theta)
    hh_velocity = hh_vec.to_beta3()
    higgs1_vec = higgs1_vec.boost_beta3(-hh_velocity)
    # print("px",higgs1_vec.px)
    # print("theta",higgs1_vec.theta)
    return abs(np.cos(higgs1_vec.theta))


In [120]:

higgs1_vec = ak.zip(
    {
        "pt": events.higgs1_reco_pt,
        "eta": events.higgs1_reco_eta,
        "phi": events.higgs1_reco_phi,
        "mass": events.higgs1_reco_mass,
    },
    with_name="Momentum4D",
)
higgs2_vec = ak.zip(
    {
        "pt": events.higgs2_reco_pt,
        "eta": events.higgs2_reco_eta,
        "phi": events.higgs2_reco_phi,
        "mass": events.higgs2_reco_mass,
    },
    with_name="Momentum4D",
)
print(higgs1_vec, type(higgs1_vec))
print(higgs1_vec.px)
print(higgs2_vec.px)
hh_vec = higgs1_vec + higgs2_vec
print(hh_vec.px, '\n\n')

higgs1_jet1_vec = ak.zip(
    {
        "pt": events.higgs1_reco_jet1_pt,
        "eta": events.higgs1_reco_jet1_eta,
        "phi": events.higgs1_reco_jet1_phi,
        "mass": events.higgs1_reco_jet1_mass,
    },
    with_name="Momentum4D",
)
higgs1_jet2_vec = ak.zip(
    {
        "pt": events.higgs1_reco_jet2_pt,
        "eta": events.higgs1_reco_jet2_eta,
        "phi": events.higgs1_reco_jet2_phi,
        "mass": events.higgs1_reco_jet2_mass,
    },
    with_name="Momentum4D",
)
higgs2_jet1_vec = ak.zip(
    {
        "pt": events.higgs2_reco_jet1_pt,
        "eta": events.higgs2_reco_jet1_eta,
        "phi": events.higgs2_reco_jet1_phi,
        "mass": events.higgs2_reco_jet1_mass,
    },
    with_name="Momentum4D",
)
higgs2_jet2_vec = ak.zip(
    {
        "pt": events.higgs2_reco_jet2_pt,
        "eta": events.higgs2_reco_jet2_eta,
        "phi": events.higgs2_reco_jet2_phi,
        "mass": events.higgs2_reco_jet2_mass,
    },
    with_name="Momentum4D",
)

higgs1_jet1_vec=ak.with_field(higgs1_jet1_vec, events.jet_pnet_ptres[:, 0], "ptres")
print(higgs1_jet1_vec.ptres)
higgs1_jet2_vec=ak.with_field(higgs1_jet2_vec, events.jet_pnet_ptres[:, -2], "ptres")
print(higgs1_jet2_vec.ptres)
higgs2_jet1_vec=ak.with_field(higgs2_jet1_vec, events.jet_pnet_ptres[:, -3], "ptres")
print(higgs2_jet1_vec.ptres)
higgs2_jet2_vec=ak.with_field(higgs2_jet2_vec, events.jet_pnet_ptres[:, -1], "ptres")
print(higgs2_jet2_vec.ptres)


[{pt: 115, eta: -0.585, phi: 0.511, mass: 128}, ... phi: 2.76, mass: 134}] <class 'vector.backends.awkward.MomentumArray4D'>
[99.9, 85.4, -166]
[19.8, 15.2, 105]
[120, 101, -60.1] 


[0.086, 0.0826, 0.0833]
[0.111, 0.117, 0.12]
[0.0744, 0.135, 0.0626]
[0.193, 0.242, 0.194]


In [121]:

higgs1_helicityCosTheta = abs(HelicityCosTheta(higgs1_vec, higgs1_jet1_vec))
higgs2_helicityCosTheta = abs(HelicityCosTheta(higgs2_vec, higgs2_jet1_vec))
hh_CosThetaStar_CS = abs(Costhetastar_CS(higgs1_vec, hh_vec))
print(higgs1_helicityCosTheta)
print(higgs2_helicityCosTheta)
print(hh_CosThetaStar_CS)
print(events.higgs1_helicityCosTheta)
print(events.higgs2_helicityCosTheta)
print(events.hh_CosThetaStar_CS)


[0.117, 0.189, 0.714]
[0.0587, 0.961, 0.9]
[0.405, 0.57, 0.737]
[0.117, 0.189, 0.714]
[0.0587, 0.961, 0.9]
[0.405, 0.57, 0.737]


In [122]:

def get_sigma_mbb(jet1, jet2):
    jet1_up=jet1*(1+jet1.ptres)
    jet2_up=jet2*(1+jet2.ptres)

    jet1_down=jet1*(1-jet1.ptres)
    jet2_down=jet2*(1-jet2.ptres)


    jet1_up_sigma=ak.singletons(abs((jet1+jet2).mass-(jet1_up+jet2).mass))
    jet1_down_sigma=ak.singletons(abs((jet1+jet2).mass-(jet1_down+jet2).mass))
    # print(jet1_up_sigma)
    # print(jet1_down_sigma)
    jet1_sigma_conc=ak.concatenate((jet1_up_sigma,jet1_down_sigma), axis=1)
    # print(jet1_sigma_conc)
    sigma_hbbCand_A=ak.max(jet1_sigma_conc, axis=1)
    # print(np.sqrt(sigma_hbbCand_A**2))

    jet2_up_sigma=ak.singletons(abs((jet1+jet2).mass-(jet1+jet2_up).mass))
    jet2_down_sigma=ak.singletons(abs((jet1+jet2).mass-(jet1+jet2_down).mass))
    # print(jet2_up_sigma)
    # print(jet2_down_sigma)
    jet2_sigma_conc=ak.concatenate((jet2_up_sigma,jet2_down_sigma), axis=1)
    # print(jet2_sigma_conc)
    sigma_hbbCand_B=ak.max(jet2_sigma_conc, axis=1)
    # print(np.sqrt(sigma_hbbCand_B**2))

    return np.sqrt(sigma_hbbCand_A**2+sigma_hbbCand_B**2)

In [123]:
sigma_mbb_norm_higgs1=get_sigma_mbb(higgs1_jet1_vec, higgs1_jet2_vec)/higgs1_vec.mass
print(sigma_mbb_norm_higgs1)
print(events.sigma_over_higgs1_reco_mass)

sigma_mbb_norm_higgs2=get_sigma_mbb(higgs2_jet1_vec, higgs2_jet2_vec)/higgs2_vec.mass
print("\n",sigma_mbb_norm_higgs2)
print(events.sigma_over_higgs2_reco_mass)

[0.0717, 0.0724, 0.0748]
[0.11, 0.132, 0.0748]

 [0.108, 0.147, 0.107]
[0.0682, 0.092, 0.107]


In [124]:
raise SystemExit

SystemExit: 

# degug

In [ ]:
boosted = higgs1_vec.boost(higgs1_vec.to_beta3())

print(higgs1_vec.to_beta3()[0])
print(boosted[0])
print(higgs1_vec[0])
print(type(higgs1_vec))
print(higgs1_vec.E[0])
print(np.sqrt(higgs1_vec.px[0]**2+higgs1_vec.py[0]**2))


{rho: 0.617, phi: 0.511, eta: -0.585, pt: 115, mass: 128}
{x: 290, y: 163, z: -206, tau: 128, pt: 115, mass: 128}
{pt: 115, eta: -0.585, phi: 0.511, mass: 128}
<class 'vector.backends.awkward.MomentumArray4D'>
185.758544921875
114.53314961865229


In [ ]:
v = ak.zip(
    {
        "pt": 1,
        "eta": 0,
        "phi": 0,
        "mass": 1,
    },
    with_name="Momentum4D",
)
print(v.px)
print(v.py)
print(v.pz)
print(v.gamma)
v1=v.boost(-v)
print(v1.px)
print(v1.py)
print(v1.y)
print(v1.pz)
print(v1.z)
print(v1)

1.0
0.0
0.0
1.4142135623730951
6.661338147750939e-16
-1.2246467991473522e-16
-1.2246467991473522e-16
0.0
0.0
{x: 6.66e-16, y: -1.22e-16, z: 0, tau: 1, pt: 1, mass: 1}


In [ ]:
v1=v.boost(v)
print(v1.px)
print(v1.py)
print(v1.y)
print(v1.pz)
print(v1.z)
print(v1.gamma)
print(v1)

2.8284271247461903
0.0
0.0
0.0
0.0
3.0000000000000004
{x: 2.83, y: 0, z: 0, tau: 1, pt: 1, mass: 1}


In [ ]:
p1=ak.zip(
    {
        "E": 10,
        "px": 3,
        "py": 4,
        "pz": 0,
    },
    with_name="Momentum4D",
)
p2=ak.zip(
    {
        "E": 8,
        "px": -2,
        "py": -1,
        "pz": 5,
    },
    with_name="Momentum4D",
)
print(p1.px)
print(p1.py)
print(p1.pt)
print(p1.mass,np.sqrt(75))
p2_boosted=p2.boost(-p1)
print(p2_boosted)

3
4
5.0
8.660254037844387 8.660254037844387
{x: -2.19, y: -1.25, z: 5, t: -8.08, E: 8, px: -2, py: -1, pz: 5}


In [ ]:
higgs2_vec = ak.zip(
    {
        "pt": events.higgs2_reco_pt,
        "eta": events.higgs2_reco_eta,
        "phi": events.higgs2_reco_phi,
        "mass": events.higgs2_reco_mass,
    },
    with_name="Momentum4D",
)
higgs2_vec_alt = ak.zip(
    {
        "px": events.higgs2_reco_pt*np.cos(events.higgs2_reco_phi),
        "py": events.higgs2_reco_pt*np.sin(events.higgs2_reco_phi),
        "pz": events.higgs2_reco_pt*np.sinh(events.higgs2_reco_eta),
        "E": np.sqrt(events.higgs2_reco_pt**2+(events.higgs2_reco_pt*np.sinh(events.higgs2_reco_eta))**2+events.higgs2_reco_mass**2),
    },
    with_name="Momentum4D",
)

print(higgs2_vec.px)

print(higgs2_vec_alt.px)
print(higgs2_vec.py)
print(higgs2_vec_alt.py)
print(higgs2_vec.E)
print(higgs2_vec_alt.E)


print(-np.log(np.tan(higgs2_vec.theta/2)))
print(events.higgs2_reco_eta)


[19.8, 15.2]
[19.8, 15.2]
[95.3, -104]
[95.3, -104]
[190, 241]
[190, 241]
[-0.968, 1.36]
[-0.968, 1.36]


In [ ]:
opp_higgs2_vec=-higgs2_vec
print(higgs2_vec)
print(opp_higgs2_vec)

print(higgs2_vec.px)
print(opp_higgs2_vec.px)
print(higgs2_vec.py)
print(opp_higgs2_vec.py)
print(higgs2_vec.pz)
print(opp_higgs2_vec.pz)
print(higgs2_vec.E)
print(opp_higgs2_vec.E)
print(higgs2_vec.theta)
print(opp_higgs2_vec.theta)
print(higgs2_vec.phi)
print(opp_higgs2_vec.phi)
print(higgs2_vec.mass)
print(opp_higgs2_vec.mass)


[{pt: 97.3, eta: -0.968, phi: 1.37, mass: 121}, ... phi: -1.43, mass: 102}]
[{rho: 97.3, phi: -1.78, eta: 0.968, tau: -121, pt: 97.3, ... pt: 105, mass: 102}]
[19.8, 15.2]
[-19.8, -15.2]
[95.3, -104]
[-95.3, 104]
[-110, 192]
[110, -192]
[190, 241]
[83.2, 193]
[2.42, 0.501]
[0.726, 2.64]
[1.37, -1.43]
[-1.78, 1.72]
[121, 102]
[-121, -102]


In [ ]:
velocity=higgs2_vec.to_beta3()
print(velocity[0])
print(velocity.x)
print(velocity.px)

velocity=(-higgs2_vec).to_beta3()
print(velocity[0])
print(velocity.x)
print(velocity.px)


print(higgs2_vec.x)
print(higgs2_vec.px)
print(higgs2_vec.gamma)

vx=higgs2_vec.px/higgs2_vec.E
print(vx)

{rho: 0.512, phi: 1.37, eta: -0.968, pt: 97.3, mass: 121}
[0.104, 0.0629]
[0.104, 0.0629]
{rho: 1.17, phi: -1.78, eta: 0.968, pt: 97.3, mass: 121}
[-0.238, -0.0784]
[-0.238, -0.0784]
[19.8, 15.2]
[19.8, 15.2]
[1.57, 2.37]
[0.104, 0.0629]
